# Explore data

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import zipfile
import os
import cv2

In [2]:
sr_91_dir = 'SR_91_Bottleneck_B_150-200'
zone_readings_path = sr_91_dir + '/Zone_Readings_0907-0908.csv'
station_sr91_path = sr_91_dir + '/Inventory.csv'

In [3]:
zone_readings = pd.read_csv(zone_readings_path)
station_sr91 = pd.read_csv(station_sr91_path)

In [4]:
# station_sr91_gdf = gpd.GeoDataFrame(station_sr91, geometry=gpd.points_from_xy(station_sr91.longitude, station_sr91.latitude))
# station_sr91_gdf.crs = {'init': 'epsg:4326'}
# station_sr91_gdf.to_file(sr_91_dir + '/Inventory.shp')

# # Compress the shapefile
# with zipfile.ZipFile(sr_91_dir + '/Inventory.zip', 'w') as z:
#     z.write(sr_91_dir + '/Inventory.shp')
#     z.write(sr_91_dir + '/Inventory.cpg')
#     z.write(sr_91_dir + '/Inventory.dbf')
#     z.write(sr_91_dir + '/Inventory.prj')
#     z.write(sr_91_dir + '/Inventory.shx')


In [5]:
zone_readings.columns

Index(['zone_id', 'measurement_start', 'speed', 'volume', 'occupancy',
       'quality'],
      dtype='object')

In [6]:
# rename 'measurement_start' to 'datetime'
zone_readings.rename(columns={'measurement_start': 'datetime'}, inplace=True)

In [7]:
# Join the zone_readings and station_sr91, based on the 'zone_id' column
zone_readings = zone_readings.merge(station_sr91, on='zone_id')
zone_readings = zone_readings[['zone_id','datetime', 'speed', 'latitude', 'longitude']]

In [8]:
# Create a GeoDataFrame from the zone_readings
zone_readings_gdf = gpd.GeoDataFrame(zone_readings, geometry=gpd.points_from_xy(zone_readings.longitude, zone_readings.latitude))

In [9]:
# Change datetime to datetime object, ignore the timezone
zone_readings_gdf['datetime'] = pd.to_datetime(zone_readings_gdf['datetime'])
zone_readings_gdf['datetime'] = zone_readings_gdf['datetime'].dt.floor('T')
zone_readings_gdf.head(5)

/var/folders/6g/xqgvnv4s1h95kfz0bd14h3_h0000gn/T/ipykernel_65224/332595671.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  zone_readings_gdf['datetime'] = zone_readings_gdf['datetime'].dt.floor('T')


,zone_id,datetime,speed,latitude,longitude,geometry
0,3262,2017-09-07 00:00:00-04:00,9.909091,27.764868,-80.954325,POINT (-80.95432 27.76487)
1,3262,2017-09-07 00:01:00-04:00,12.238095,27.764868,-80.954325,POINT (-80.95432 27.76487)
2,3262,2017-09-07 00:02:00-04:00,13.565217,27.764868,-80.954325,POINT (-80.95432 27.76487)
3,3262,2017-09-07 00:03:00-04:00,21.894737,27.764868,-80.954325,POINT (-80.95432 27.76487)
4,3262,2017-09-07 00:04:00-04:00,27.585366,27.764868,-80.954325,POINT (-80.95432 27.76487)


# Use the plot

In [10]:
# Generate time samples, in 'US/Eastern'.
time_samples = pd.date_range(start='2017-09-07 00:00:00', end='2017-09-08 23:59:00', freq='5min', tz='US/Eastern')
# time_samples

# Make directory for images
os.makedirs('img', exist_ok=True)

for time_sample in time_samples:
    # Filter the zone_readings_gdf for the time_sample
    zone_readings_gdf_sample = zone_readings_gdf[zone_readings_gdf['datetime'] == time_sample]

    # Create plot for this sample
    fig, ax = plt.subplots(figsize=(10, 10))
    zone_readings_gdf_sample.plot(column='speed', markersize=5, cmap='RdYlGn', ax=ax)
    ax.set_title('Speeds at ' + str(time_sample))
    plt.savefig('img/' + str(time_sample) + '.png')
    plt.close(fig)

In [11]:
# Define the directory containing the images
img_dir = 'img'

# Get a list of all image files in the directory
images = [img for img in os.listdir(img_dir) if img.endswith(".png")]
images.sort()  # Ensure the images are in the correct order

# Define the video file name and frame rate
video_file = 'output_video.mp4'
frame_rate = 5  # 1 frame per second

# Read the first image to get the dimensions
first_image_path = os.path.join(img_dir, images[0])
frame = cv2.imread(first_image_path)
height, width, layers = frame.shape

# Define the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_file, fourcc, frame_rate, (width, height))

# Add each image to the video
for image in images:
    image_path = os.path.join(img_dir, image)
    frame = cv2.imread(image_path)
    video.write(frame)

# Release the video writer
video.release()

print(f"Video saved as {video_file}")

Video saved as output_video.mp4
